In [ ]:
%matplotlib inline
import skyflux as sf

Missing flux value for: GLEAM J202818-254302 at frequency: 204 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 212 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 220 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 227 MHz.
Missing spectral index for: GLEAM J202818-254302


//anaconda3/envs/hera/lib/python3.8/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.8/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.8/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.8/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.8/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp

In [ ]:
# Display available antennae
print(sf.demo.active_ants)

In [ ]:
min_residual = 100
target = sf.rot.hera_lat
best_i = -1
for i in range(len(sf.catalog.obj_catalog)):
    obj = sf.catalog.obj_catalog[i]
    residual = (obj.dec_angle - target) ** 2
    if residual < min_residual:
        min_residual = residual
        best_i = i
print(best_i)
print(sf.catalog.obj_catalog[best_i])
best_source = sf.catalog.obj_catalog[best_i]
print("Flux at 76MHz:", best_source.flux_by_frq[76])
# also, would we not expect the first term to dominate? And yet it is not even the largest
    # of numbers all with magnitudes less than one

In [ ]:
#S_nu \propto \nu^\alpha
flx = best_source.flux_by_frq

In [ ]:
cop = lambda source, nu : source.flux_by_frq[nu / 1e6] / nu ** source.alpha

cops = np.array([cop(best_source, nu * 1e6) for nu in flx])

plt.title("Spectral Coefficient of Proportionality")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Coefficient [?]")
plt.plot(flx.keys(), cops)

In [ ]:
ra = np.radians(best_source.ra_angle)
dec = np.radians(best_source.dec_angle)
dec = np.radians(sf.rot.hera_lat)

r = sf.rot.radec2lm(ra, dec, ra0=0)

print(r)

print(sf.ant.baseline(88, 121)[0:2])

In [ ]:
this_time = np.radians(best_source.ra_angle)

print("Using time:", this_time)

sf.vis.visibility(88, 121, best_source, nu=151e6,
                  time=this_time)[0]

In [ ]:
vist = sf.vis.sources_over_frequency(
    88, 121, best_source, start=76e6, end=227e6, time=this_time
)
#vist

In [ ]:
I = np.array([v[1][0] for v in vist])
nu_axis = vist[:, 0] / 1e6

plt.plot(nu_axis, np.real(I), label="I, real")
plt.plot(nu_axis, np.imag(I), label="I, imaginary")
plt.plot(nu_axis, np.abs(I), label="I, magnitude")

plt.xlabel("Frequency [MHz]")
plt.ylabel("Visibility [Jy?]")

plt.legend()
plt.show()

In [ ]:
Q = np.array([v[1][1] for v in vist])
U = np.array([v[1][2] for v in vist])
V = np.array([v[1][3] for v in vist])

plt.plot(nu_axis, np.real(I), label="I")
plt.plot(nu_axis, np.real(Q), label="Q")
plt.plot(nu_axis, np.real(U), label="U")
plt.plot(nu_axis, np.real(V), label="V")

plt.xlabel("Frequency [MHz]")
plt.ylabel("Visibility [Jy?]")
plt.title("Real components")

plt.legend()
plt.show()

In [ ]:
plt.plot(nu_axis, np.imag(I), label="I")
plt.plot(nu_axis, np.imag(Q), label="Q")
plt.plot(nu_axis, np.imag(U), label="U")
plt.plot(nu_axis, np.imag(V), label="V")

plt.xlabel("Frequency [MHz]")
plt.ylabel("Visibility [Jy?]")
plt.title("Imaginary components")

plt.legend()
plt.show()

In [ ]:
plt.plot(nu_axis, np.angle(I), label="I, angle")

plt.xlabel("Frequency [MHz]")
plt.ylabel("Angle [rad]")

plt.legend()
plt.show()

In [ ]:
nu_list = [1e6 * i for i in range(76, 100)]

ra = np.radians(best_source.ra_angle)
dec = np.radians(best_source.dec_angle)

# constants for the LST plot
print("Right ascension:", ra)
print("Frequency interval", 1e6)
print("Declination:", dec)
print("Reference point:", np.radians(-30.72))

list_A = []
list_J = []

for nu in nu_list:
    J = sf.stokes.create_J(ra=ra, dec=dec,
                           lst=this_time, nu=nu, radians=True)
    list_J.append(J)
    A = sf.stokes.create_A(J=J)
    list_A.append(A)

list_A = np.array(list_A)
list_J = np.array(list_J)
    
print(list_A[0][0][0])

plt.plot(nu_list, np.real(list_A[:, 0, 0, 0]), label="I' <- I, real")
plt.plot(nu_list, np.abs(list_J[:, 0, 0, 0]), label="J: ||xx||")
plt.plot(nu_list, np.imag(list_A[:, 0, 0, 0]), label="I' <- I, imag")

plt.xlabel("LST [rad]")
plt.ylabel("Visibility [Jy?]")

plt.legend()
plt.show()

In [ ]:
print("The following is the universal baseline")
print(sf.ant.baseline(88, 121))

In [ ]:
len(sf.catalog.obj_catalog)

In [ ]:
# Is the frequency making a difference for J and A?
az, alt = sf.rot.eq_to_topo(
    best_source.ra_angle, best_source.dec_angle,
    lst=best_source.ra_angle, radians=False)

az = np.array([np.radians(az)])
alt = np.array([np.radians(alt)])

# format_J has nothing to do with frequency, so theoretically this should directly
# inform us about the influence of the RIMEz code on the Jones matrix
def direct_J(nu):
    return sf.stokes.format_J(sf.stokes.spline_beam_func(nu, alt, az))

In [ ]:
direct_J(np.array([151e6]))

In [ ]:
print(direct_J(np.array([50e6])))
print(direct_J(np.array([51e6])))
print(direct_J(np.array([52e6])))

print(direct_J(50e6))

In [ ]:
sf.stokes.beam_frqs

In [ ]:
#sf.stokes.spline_beam_func(np.array([0]), np.array([0]), np.array([0]))